In [3]:
!pip install transformers requests beautifulsoup4 

  Using cached transformers-4.41.2-py3-none-any.whl.metadata (43 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     -------------------------------------- 42.0/42.0 kB 503.7 kB/s eta 0:00:00
  Using cached tqdm-4.66.4-py3-none-any.whl.metadata (57 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
  Using cached fsspec-2024.6.0-py3-none-any.whl.metadata (11 kB)
Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached huggingface_hub-0.23.4-py3-none-any.whl (402 kB)
   ---------------------------------------- 0.0/269.0 kB ? eta -:--:--
   ------------ --------------------------- 81.9/269.0 kB 1.5 MB/s eta 0:00:01
   ------------------------- -----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1 requires mkl<=2021.4.0,>=2021.1.1; platform_system == "Windows", which is not installed.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

c:\Users\Aman\.conda\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer= AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [49]:
tokens=tokenizer.encode('It was awesome but could have been more satisfying.', return_tensors='pt')


In [40]:
tokens[0]

tensor([  101,   151, 10345,   143, 14628, 11633,   106,   102])

In [41]:
tokenizer.decode(tokens[0])

'[CLS] i am a sadist! [SEP]'

In [50]:
result=model(tokens)

In [51]:
result.logits

tensor([[-2.7282, -1.1212,  1.5814,  1.9052,  0.2150]],
       grad_fn=<AddmmBackward0>)

In [52]:
int(torch.argmax(result.logits))+1

4

In [4]:
r=requests.get('https://www.imdb.com/title/tt27470893/reviews')
soup=BeautifulSoup(r.text, 'html.parser')
regex=re.compile('.*text show-more__control.*')
results=soup.find_all('div', {'class':regex})
reviews=[result.text for result in results]

In [5]:
reviews[0]

'I recently watched "Chandu Champion" and I absolutely loved it! This film is a remarkable portrayal of a real-life hero and it left a lasting impression on me.The movie tells the incredible true story of Murlikant Petkar, a parathlete who overcame immense obstacles to become India\'s first Paralympic gold medalist. The narrative is deeply inspiring, shedding light on the triumph of the human spirit and the power of perseverance.The performances in the film were outstanding. The lead actor, Kartik Aaryan, delivered a powerful and moving performance as Murlikant Petkar. His portrayal was nuanced and heartfelt, capturing the essence of Petkar\'s struggles and victories with great authenticity. The supporting cast also did an excellent job, adding depth and richness to the story.A special mention must go to the director, Kabir Khan, who brought this inspiring story to life with such skill and sensitivity. The direction was masterful, balancing emotional depth with engaging storytelling. T

In [59]:
pip install pandas

  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB 1.4 MB/s eta 0:00:09
   ---------------------------------------- 0.1/11.6 MB 1.2 MB/s eta 0:00:10
   ---------------------------------------- 0.1/11.6 MB 1.0 MB/s eta 0:00:12
    --------------------------------------- 0.2/11.6 MB 1.1 MB/s eta 0:00:11
    --------------------------------------- 0.2/11.6 MB 1.1 MB/s eta 0:00:11
    --------------------------------------- 0.3/11.6 MB 1.1 MB/s eta 0:00:11
   - -------------------------------------- 0.3/11.6 MB 1.1 MB/s eta 0:00:11
   - -------------------------------------- 0.4/11.6 MB 1.1 MB/s eta 0:00:11
   - -------------------------------------- 0.4/11.6 MB 1.1 MB/s eta 0:00:11
   - -------------------------------------- 0.5/11.6 MB 1.1 MB/s eta 0:00:11
   - ----------------------

In [7]:
import pandas as pd
import numpy as np

In [8]:
df=pd.DataFrame(np.array(reviews), columns=['review'])

In [9]:
df.tail()

,review
20,I just saw Chandu Champion and it was good. Ge...
21,Murlikant Petkar is such a personality who onl...
22,This is such a great movie for Indians. I hope...
23,"""Chandu Champion"" is a heartwarming tale of Mu..."
24,"This story deserves to be heard, watched, and ..."


In [10]:
def senti_score(review):
    tokens=tokenizer.encode(review, return_tensors='pt')
    result=model(tokens)
    return int(torch.argmax(result.logits))+1

In [11]:
senti_score(df['review'].iloc[9])

4

In [12]:
df['sentiment']=df['review'].apply(lambda x: senti_score(x[:512]))  ## this transformer nlp model can only recognise first 512 characters


In [13]:
df

,review,sentiment
0,"I recently watched ""Chandu Champion"" and I abs...",5
1,"This is one of the finest movies of Bollywood,...",5
2,It is not just a movie nor a documentary.Karti...,5
3,Guys please visit and watch the heartwarming m...,5
4,"Chandu Champion,"" directed by Kabir Khan, is a...",5
5,I didn't go in the theatre with a lot of expec...,5
6,*** Chandu Champion Hindi Movie Review ***Dire...,4
7,"""Chandu Champion"" is a masterful blend of spor...",4
8,I genuinely urge you to catch this movie in th...,5
9,Powerfully story as it taken from a true story...,4


In [17]:
df['review'][8]

"I genuinely urge you to catch this movie in theaters for an incredible experience that promises not just entertainment but also profound inspiration and motivation. I attended the first-night show and was completely swept away-despite not shedding a tear all year, I found myself moved to tears four times during the film. It's a powerful story that touches the heart and soul, resonating deeply with its emotional depth and compelling narrative. The theater ambiance amplifies the emotions, making it a truly immersive and unforgettable cinematic journey. Don't miss out on this opportunity to be moved, inspired, and captivated by a film that delivers on every level."